In [2]:
import requests
from bs4 import BeautifulSoup
import random
import concurrent.futures
import time
from datetime import datetime
from userAgents import users
from nordvpn_switcher import initialize_VPN, rotate_VPN, terminate_VPN
from twilio.rest import Client
import requests
import time

#Twilio Information
account_sid = "some token"
auth_token = "some token"

# Creates a client object
client = Client(account_sid, auth_token)

def sendText(msg): 
    '''
    Send SMS text to my phone
    '''
    client.messages.create(
        to ="some phone number",
        from_ = "some phone number",
        body = str(msg))

file = open('log' + '.txt', 'a+')
referers = ['https://www.amazon.com/','https://www.facebook.com/','https://www.instagram.com/','https://www.nvidia.com/en-us/','https://www.walmart.com/','https://www.bestbuy.com/',
                        'https://www.target.com/','https://www.bhphotovideo.com/','https://www.adorama.com/','https://www.newegg.com/','https://www.google.com/','https://www.asus.com/us/',
                        'https://www.amd.com/en','https://www.apple.com/','https://www.dell.com/en-us']

def consoleLog(string):
    file.write(string)
    print(string)

def login():
    '''define URL where login form is located'''
    site = 'https://www.amazon.com/gp/sign-in.html'

    '''initiate session'''
    session = requests.Session()

    '''define session headers'''
    session.headers = {
    'User-Agent': random.choice(users),
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'Referer': random.choice(referers)
    }

    '''get login page'''
    resp = session.get(site)
    html = resp.text

    '''get BeautifulSoup object of the html of the login page'''
    soup = BeautifulSoup(html , 'lxml')

    '''scrape login page to get all the needed inputs required for login'''
    data = {}
    form = soup.find('form', {'name': 'signIn'})
    for field in form.find_all('input'):
        try:
            data[field['name']] = field['value']
        except:
            pass

    '''add username and password to the data for post request'''
    data[u'email'] = 'Test@email.com'
    data[u'password'] = 'Some Password'

    '''submit post request with username / password and other needed info'''
    post_resp = session.post('https://www.amazon.com/ap/signin', data = data)

    post_soup = BeautifulSoup(post_resp.content , 'lxml')

    if post_soup.find_all('title')[0].text == 'Your Account':

        session.headers = {
        'User-Agent': random.choice(users),
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Referer': site
    }
        return session, True
    else:
        return session, False

def addToCart(asin,session):
    
    payload = {
        'ASIN': asin,
        'submit.buy-now': 'Submit'
    }

    data = session.post('https://www.amazon.com/gp/product/handle-buy-box/ref=dp_start-bbf_1_glance', data = payload)
    
    added = BeautifulSoup(data.text , 'html.parser')
    added = added.findAll('h1',{'class':'a-spacing-mini a-spacing-top-base'})

    if len(added) == 0:
        return True
    else:
        return False

def autoCheckout(asin,session):
    
    data = session.get("https://www.amazon.com/gp/buy/spc/handlers/display.html?hasWorkingJavascript=1")
    
    form = BeautifulSoup(data.text , 'lxml')
    
    if data.ok == True:
        
        autoCheckoutData = {}
        formData = form.findAll("input")
        for data in formData:
            try:
                autoCheckoutData[data['name']] = data['value']
            except:
                pass

        autoCheckoutData['hasWorkingJavascript'] = 1
        autoCheckoutData['placeYourOrder1'] = 1
        autoCheckoutData['promiseAsin-0'] = asin

        session.post("https://www.amazon.com/gp/buy/spc/handlers/static-submit-decoupled.html/ref=ox_spc_place_order?ie=UTF8&hasWorkingJavascript=", data = autoCheckoutData)
        
        return True
    
    else:
        return False

def randomSleep():

    num1 = random.randint(1,3)
    num2 = random.randint(1,3)
    num3 = random.randint(1,3)
    num4 = random.randint(1,3)

    if num1 == num2 and num1 == num3 and num1 == num4:
        sleepSec = (random.randint(1000,3000) / 1000)
        consoleLog("RANDOM SLEEP: Sleeping for " + str(sleepSec) + " seconds.\n")
        time.sleep(sleepSec)
        
def getPrice(asin):
    
    while True:
        '''define session headers'''
        headers = {
        'User-Agent': random.choice(users),
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'Referer': 'https://www.amazon.com/'
        }

        try:
            checkPrice = requests.get(url = ('https://www.amazon.com/dp/' + asin), headers = headers, timeout = 10)  
            html = BeautifulSoup(checkPrice.text, 'html.parser')
            if checkPrice.ok == False:
                return -1
        except:
            return -1

        i = 0
        priceText = None

        while i != 5 and priceText == None:

            i += 1

            try:
                if i == 1:
                    priceText = html.find('span',{'id':'price_inside_buybox'})
                elif i == 2:
                    priceText = html.find('span',{'id':'newBuyBoxPrice'})
                elif i == 3:
                    priceText = html.find('span',{'id':'priceblock_ourprice'})
                elif i == 4:
                    priceText = html.find('span',{'id':'price'})
                elif i == 5:
                    priceText = html.find('span',{'id':'priceblock_dealprice'})
                else:
                    consoleLog("Something is wrong.")
            except:
                pass

        try:
            priceText = priceText.text
            priceText = priceText.replace("$","")
            priceText = priceText.replace(",","")
            priceFloat = float(priceText)
            return priceFloat
        except:
            if checkPrice.ok == True:
                pass
            else:
                return -1

def run(asin):
    loop = 10
    inCart = False
    checkout = False
    
    terminate_VPN(settings)
    time.sleep(1)
    session, status = login()
    
    if status == False:
        consoleLog("Could not login.")
    else:
        consoleLog("Logged in.")

    while inCart == False and loop != 0 and status == True:
        loop -= 1
        consoleLog("Attempting to add item to cart...")
        inCart = addToCart(asin,session)
    
    if inCart == True: 
        consoleLog("Attempting to checkout...")
        checkout = autoCheckout(asin,session)
    else:
        consoleLog("Could not add to cart.")
        
    session.close()
    
    if checkout == True:
        return True
    else:
        return False

def main():

    attempt = 0 

    consoleLog("\nProgram Started...\n")
    
    gpu3060 = ['B091H2KFDH','B08WHJPBFX','B08WGTL4CW','B08XNLLPY7','B08WT47L8B',
               'B08X4VJ6SB','B08WRQ3JR1','B091MNBNWT','B08ZZW34T3','B08WPJ5P4R',
               'B08WM28PVH','B08X4V8N5Q','B0916KXJJB','B08W8DGK3X']
    
    gpu3060ti = ['B08P3V572B','B08NYPLXPJ','B08P2D3JSG','B08PW13YB1','B08P2HBBLX',
               'B083Z7TR8Z','B08R6QRZ9D','B08P2DQ28S','B08S6Z5285','B08P3XJLJJ',
               'B08QZ5GJ52','B08R876RTH','B08P2H5LW2','B083Z5P6TX']

    testItem = ['B071JM699B']

    asins = []
    asins += gpu3060ti
    
    stop = False
    while stop == False: 

        #randomSleep()   
        priceList = []

        # runs code
        start = time.perf_counter()
        with concurrent.futures.ThreadPoolExecutor() as executor:
               prices = executor.map(getPrice,asins)

        for price in prices:
            priceList.append(price)
        
        end = time.perf_counter()

        duration = round(end-start,2)
        if duration < 2 or (-1 in priceList):
            
            print("Changing IP...")
            rotate_VPN(settings)
            
        attempt += 1
        consoleLog("Attempt: " + str(attempt) + " // Duration: " + str(duration) + " // Current Time: " + str(datetime.now()))
        consoleLog("Prices: " + str(priceList) + "\n")
        
        for price in priceList:
            
            if price > 300 and price < 800:
                start = time.perf_counter()
                if (run(asins[(priceList.index(price))])):
                    stop = True
                    end = time.perf_counter()
                    print("Time spent executing order: " + str(round(end-start,2)) + "\n")
                    break
                else:
                    end = time.perf_counter()
                    print("Time spent executing order: " + str(round(end-start,2)) + "\n")
                    consoleLog("Could not checkout.")
                
        random.shuffle(asins)
        
    consoleLog("Item Purchased!")
    return True

In [ ]:
while True:
    try:
        settings = initialize_VPN(area_input=['United States', 'Canada','United Kingdom'])
        rotate_VPN(settings)
        if(main()):
            sendText("I did something!\n" + str(datetime.now()))
            consoleLog("Running again!")
    except:
        consoleLog("Code Error. Changing IP.\n")
        rotate_VPN(settings)
        
consoleLog("Program ended")

In [ ]:
# start = time.perf_counter()
# with concurrent.futures.ThreadPoolExecutor() as executor:
#        prices = executor.map(getPrice,asins)

# for price in prices:
#     priceList.append(price)

# end = time.perf_counter()